## Math

In [ ]:
# ===================================================
# 0. 설치 및 기본 설정
# ===================================================
!pip install transformers datasets --quiet

import re, torch, random
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import pipeline

# ===================================================
# 1. 데이터셋 로딩 및 샘플링
# ===================================================
print("🔄 Loading and sampling MathX-5M...")
streaming_dataset = load_dataset("XenArcAI/MathX-5M", split="train", streaming=True)
subset = list(streaming_dataset.take(10000))  # Random 10,000

# ===================================================
# 2. LLM으로 학생 답변 생성 (OpenAI 또는 Gemma 가능)
# ===================================================
# GPT API 버전 예시 (OpenAI 사용 시)
from openai import OpenAI

openai = OpenAI(api_key="YOUR_OPENAI_API_KEY")

def generate_student_answer_gpt(problem):
    prompt = f"Solve the following problem and give only the final answer: {problem}"
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return "ERROR"

# 또는 Hugging Face Pipeline 사용 시 (Gemma)
# from transformers import pipeline
# llm = pipeline("text-generation", model="google/gemma-2b-it", device_map="auto")

# def generate_student_answer(problem):
#     output = llm(f"Solve: {problem}", max_new_tokens=20)
#     return output[0]["generated_text"].split("\n")[0].strip()

# ===================================================
# 3. 정답 비교 함수
# ===================================================
def clean_number(text):
    # 수학적 숫자만 추출 (기본형)
    matches = re.findall(r"-?\d+(?:\.\d+)?", text)
    return matches[0] if matches else None

def is_correct(student_answer, expected_answer):
    try:
        sa = clean_number(student_answer)
        ea = clean_number(expected_answer)
        return int(float(sa)) == int(float(ea))
    except:
        return False

# ===================================================
# 4. 데이터셋 생성 (문제 + 답변 + 라벨)
# ===================================================
print("🧠 Generating answers and assigning labels...")
formatted = []

for sample in subset:
    problem = sample["problem"]
    expected = sample["expected_answer"]
    student = generate_student_answer_gpt(problem)

    label = 1 if is_correct(student, expected) else 0
    formatted.append({
        "input": f"Q: {problem}\nStudent Answer: {student}",
        "label": label
    })

# ===================================================
# 5. Class Balance 조정 (0/1 비율 1:1 맞추기)
# ===================================================
df = pd.DataFrame(formatted)
df = df[df["label"].isin([0,1])]

correct = df[df["label"] == 1]
wrong = df[df["label"] == 0]

min_class_size = min(len(correct), len(wrong))
balanced = pd.concat([correct.sample(min_class_size), wrong.sample(min_class_size)])

balanced = balanced.sample(frac=1).reset_index(drop=True)
print(f"✅ Balanced dataset size: {len(balanced)} (1s: {sum(balanced.label==1)}, 0s: {sum(balanced.label==0)})")

# 저장 또는 학습용 Dataset 변환
from google.colab import drive
drive.mount('/content/drive')

classification_dataset = Dataset.from_pandas(balanced)
classification_dataset.save_to_disk("/content/drive/MyDrive/Gemma_FineTuning")

## Reading/Summary

In [ ]:
# ===================================================
# 0. Install dependencies (Colab)
# ===================================================
!pip install transformers datasets sentence-transformers accelerate peft bitsandbytes trl --quiet

import torch, re
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig,
                          TrainingArguments, DataCollatorWithPadding)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# ===================================================
# 1. Load CNN/DailyMail and Sample 2000
# ===================================================
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:2000]")

# ===================================================
# 2. Load Summarization Model (T5-small for speed)
# ===================================================
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# ===================================================
# 3. Generate Student Summaries
# ===================================================
print("🔄 Generating student summaries...")
student_summaries = []

for item in dataset:
    input_text = "summarize: " + item["article"][:1500]  # truncate long articles
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=64)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    student_summaries.append(summary)

# ===================================================
# 4. Embed & Score via Similarity (0~5 scale)
# ===================================================
print("🧠 Scoring with similarity...")
scorer = SentenceTransformer("all-MiniLM-L6-v2")
examples = []

def get_score(student, reference):
    sim = cosine_similarity([scorer.encode(student)], [scorer.encode(reference)])[0][0]
    if sim > 0.85:
        return 5
    elif sim > 0.75:
        return 4
    elif sim > 0.65:
        return 3
    elif sim > 0.55:
        return 2
    elif sim > 0.45:
        return 1
    else:
        return 0

for i, item in enumerate(dataset):
    article = item["article"]
    reference = item["highlights"]
    student = student_summaries[i]
    label = get_score(student, reference)
    examples.append({
        "input": f"Article: {article}\n\nStudent Summary: {student}",
        "label": label
    })

# ===================================================
# 5. Convert to HF Dataset and Save
# ===================================================
df = pd.DataFrame(examples)
data = Dataset.from_pandas(df)

from google.colab import drive
drive.mount('/content/drive')

data.save_to_disk("/content/drive/MyDrive/Gemma_FineTuning/essay_eval_multiclass_2000")